# Merge lists of strings

## Instructions
Write an immutable function that merges the following inputs into a single list.

Inputs
- Original list of strings
- List of strings to be added
- List of strings to be removed

Return
- List shall only contain unique values
- List shall be ordered as follows
--- Most character count to least character count
--- In the event of a tie, reverse alphabetical

Other Notes
- You can use any programming language you like
- The function you submit shall be runnable

For example:

- Original List = ['one', 'two', 'three']
- Add List = ['one', 'two', 'five', 'six']
- Delete List = ['two', 'five']
- Result List = ['three', 'six', 'one']


## My answer

* Step 1: concatenate the original list and the list of elements to add, store the results in a new list
* Step 2: remove the elements in the new list that are present in the list of elements to delete. Everytime a duplicate element is encountered, add it the the list of elements to delete.
* Step 3: Sort the list. First, based on the lexicographic order. Then, based on the length of each elements. Finally, reverse the order of the list.

### Import relevant modules

In [2]:
import numpy as np   # we will use numpy to sort the list (in step 3)

### Some helper functions

The functions below will be useful for our purpose and they will make the main function "list_merge" more digest to read and comprehend.

In [3]:
# The function below returns True if all the elements of 'my_list' are strings, False otherwise
def contains_only_str(my_list):
    for item in my_list:
        if not isinstance(item, str):
            return False
    return True
    
err_msg = "Non-string element detected in "
    
# The function below adds 'my_str' to the list 'my_list' except if it creates a duplicate
def add_str(my_list, my_str):
    for item in my_list:
        if item == my_str:
            return my_list
    my_list.append(my_str)
    return my_list

# The function below removes all the elements in 'to_del' from 'my_list' as well as any duplicate
def rm_list_and_dupl(my_list, to_del):
    to_del_copy = to_del[:]
    current_length = len(my_list)
    i = 0
    while i < current_length:
        duplicate = False
        j = 0
        while (not duplicate) and (j < len(to_del_copy)):
            if my_list[i] == to_del_copy[j]:
                duplicate = True
                my_list.pop(i)
                current_length-=1
                i-=1
            j+=1
        if not duplicate:
            add_str(to_del_copy, my_list[i])
        i+=1
    return my_list

### The desired function

In [4]:
# The function below performs the desired task following the 3 steps described above
def list_merger(my_list, to_add, to_del):
    assert contains_only_str(my_list), err_msg + "my_list."
    assert contains_only_str(to_add), err_msg + "to_add."
    assert contains_only_str(to_del), err_msg + "to_del."
    new_list = my_list[:]
    
    # Step 1: add the elements from 'to_add'
    new_list = new_list + to_add
    
    # Step 2: remove the elements from 'to_del' as well as duplicates
    new_list = rm_list_and_dupl(new_list, to_del)
    
    # Step 3: sort the resulted list
        # Step 3.1: sort based on the lexicographic order
    new_list = sorted(new_list)
    
        # Step 3.2: sort based on the length of the strings
            # Step 3.2.1: convert the list to a numpy array
    new_list = np.array(new_list, dtype = 'str')
            # Step 3.2.2: Create an array containing the lengths of all the elements of 
            # 'new_list', sort it and feed the sorted indices to 'new_list' itself. In
            # effect, this sorts 'new_list' based on the length of its elements
    new_list = new_list[np.array([len(new_list[i]) for i in range(len(new_list))]).argsort()]
    
    # Step 3.3: reverse the array and convert back to a list to obtain the desired output
    new_list = np.flip(new_list).tolist()
    
    return new_list

### Some tests

#### Automatic testing

In [5]:
# The function below returns True if 'my_list' is sorted following a reverse length & alphabetical ordering
def test_ordering(my_list):
    assert contains_only_str(my_list), err_msg + "result."
    correct = True
    message = ""
    for i in range(len(my_list) - 1):
        if len(my_list[i]) < len(my_list[i+1]):
            correct = False
            message = "Sorting based on reversed length is wrong."
            break
        if len(my_list[i]) == len(my_list[i+1]):
            if (my_list[i:i+2][0] != sorted(my_list[i:i+2])[::-1][0]):
                correct = False
                message = "Sorting based on reverse alphabeltical order is wrong."
                break
    return (correct, message)

def test_content(my_list, to_add, to_del):
    assert contains_only_str(my_list), err_msg + "my_list."
    assert contains_only_str(to_add), err_msg + "to_add."
    assert contains_only_str(to_del), err_msg + "to_del."
    
    correct = True
    message = ""
    should_be_added = [True for i in range(len(to_add))]
    for i in range(len(to_add)):
        for j in range(len(to_del)):
            if to_add[i] == to_del[j]:
                should_be_added[i] = False
    for i in range(len(to_add)):
        for j in range(len(my_list)):
            if to_add[i] == my_list[j]:
                should_be_added[i] = False
        if should_be_added[i]:
            correct = False
            message = "\"" + to_add[i] + "\"  was not added."
    for i in range(len(my_list)):
        for j in range(len(to_del)):
            if my_list[i] == to_del[j]:
                correct = False
                message = "\"" + to_del[j] + "\"  was not deleted."
    return (correct, message)

def test_result(my_list, to_add, to_del):
    correct = True
    ordering = test_ordering(my_list)
    ordering = (True, "")
    content = test_content(my_list, to_add, to_del)
    if ordering[0] and content[0]:
        print("Test succeeded.")
    else:
        correct = False
        print("Test failed: ", ordering[1], content[1])
    return correct

#### Perform tests

In [9]:
a = ['one', 'two', 'three',]
b = ['one', 'two', 'five', 'six']
c = ['two', 'five']
print("Test 1:")
print("Original List =", a)
print("Add List =", b)
print("Delete List =", c)
result = list_merger(a, b, c)
print("Result List =", result)
test_result(result, b, c)
print("\n")

a = ['six', 'two', 'three', 'nine']
b = ['one', 'two', 'five', 'six', 'five', 'seven', 'four']
c = ['two']
print("Test 2:")
print("Original List =", a)
print("Add List =", b)
print("Delete List =", c)
result = list_merger(a, b, c)
print("Result list =", result)
test_result(result, b, c)
print("\n")

print("Test 3:")
a = ['six', 'two', 'three', 'nine']
c = ['one', 'two', 'five', 'six', 'five', 'seven', 'four']
b = []
print("Original List =", a)
print("Add List =", b)
print("Delete List =", c)
result = list_merger(a, b, c)
print("Result list =", result)
test_result(result, b, c)
print("\n")

print("Test 4:")
a = ['six', 'two', 'three', 'nine']
c = ['one', 'two', 'five', 'six', 'five', 'seven', 'four']
b = ['', 'four']
print("Original List =", a)
print("Add List =", b)
print("Delete List =", c)
result = list_merger(a, b, c)
print("Result list =", result)
test_result(result, b, c)
print("\n")

print("Test 5:")
a = ['six', 'two', 'three', 'nine']
b = []
c = ['', 'nine']
print("Original List =", a)
print("Add List =", b)
print("Delete List =", c)
result = list_merger(a, b, c)
print("Result list =", result)
test_result(result, b, c)
print("\n")

Test 1:
Original List = ['one', 'two', 'three']
Add List = ['one', 'two', 'five', 'six']
Delete List = ['two', 'five']
Result List = ['three', 'six', 'one']
Test succeeded.


Test 2:
Original List = ['six', 'two', 'three', 'nine']
Add List = ['one', 'two', 'five', 'six', 'five', 'seven', 'four']
Delete List = ['two']
Result list = ['three', 'seven', 'nine', 'four', 'five', 'six', 'one']
Test succeeded.


Test 3:
Original List = ['six', 'two', 'three', 'nine']
Add List = []
Delete List = ['one', 'two', 'five', 'six', 'five', 'seven', 'four']
Result list = ['three', 'nine']
Test succeeded.


Test 4:
Original List = ['six', 'two', 'three', 'nine']
Add List = ['', 'four']
Delete List = ['one', 'two', 'five', 'six', 'five', 'seven', 'four']
Result list = ['three', 'nine', '']
Test succeeded.


Test 5:
Original List = ['six', 'two', 'three', 'nine']
Add List = []
Delete List = ['', 'nine']
Result list = ['three', 'two', 'six']
Test succeeded.


